# Hate Speech Detection System

In [2]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaish\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

## Preprocessing the dataset

In [5]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.update(list(string.punctuation))


In [6]:
df = pd.read_csv("cyberbullying_tweets.csv")
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [7]:
messages = df['tweet_text']
y = df['cyberbullying_type']

In [8]:
df['tweet_text']

0        In other words #katandandre, your food was cra...
1        Why is #aussietv so white? #MKR #theblock #ImA...
2        @XochitlSuckkks a classy whore? Or more red ve...
3        @Jason_Gio meh. :P  thanks for the heads up, b...
4        @RudhoeEnglish This is an ISIS account pretend...
                               ...                        
47687    Black ppl aren't expected to do anything, depe...
47688    Turner did not withhold his disappointment. Tu...
47689    I swear to God. This dumb nigger bitch. I have...
47690    Yea fuck you RT @therealexel: IF YOURE A NIGGE...
47691    Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...
Name: tweet_text, Length: 47692, dtype: object

In [9]:
def get_simple_pos(tag) :
    if tag.startswith('J') :
        return wordnet.ADJ
    elif tag.startswith('V') :
        return wordnet.VERB
    elif tag.startswith('N') :
        return wordnet.NOUN
    elif tag.startswith('R') :
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(review) :
    global max_len
    words = word_tokenize(review)
    output_words = []
    for word in words :
        if word.lower() not in stop_words :
            pos = pos_tag([word])
            clean_word = lemmatizer.lemmatize(word,pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    max_len = max(max_len, len(output_words))
    return " ".join(output_words)

In [10]:
max_len = 0

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vaish\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [14]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vaish\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [16]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vaish\AppData\Roaming\nltk_data...


True

In [17]:
print(messages[0])
messages = [clean_text(message) for message in messages]
print(messages[0])

In other words #katandandre, your food was crapilicious! #mkr
word katandandre food crapilicious mkr


In [18]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as file:
        word_to_vec_map = {}
        word_to_index = {}
        index_to_word = {}
        index = 0
        for line in file:
            line = line.strip().split()
            curr_word = line[0]
            word_to_index[curr_word] = index
            index_to_word[index] = curr_word
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
            index += 1
    return word_to_index, index_to_word, word_to_vec_map

In [20]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [21]:
def sentences_to_indices(X, word_to_index, max_len):
    m = len(X)
    X_indices = np.zeros((m, max_len))
    for i in range(m):
        sentence_words = [w.lower() for w in X[i].split()]
        j = 0
        for word in sentence_words:
            if word in word_to_index:
                X_indices[i, j] = word_to_index[word]
            j += 1
    return X_indices

## The LSTM model

In [22]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["cucumber"].shape[0]
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [23]:
def NLPModel(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    
    embeddings = pretrained_embedding_layer(word_to_vec_map, word_to_index)(sentence_indices)
    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(1)(X)
    X = Activation('sigmoid')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [24]:
model = NLPModel((max_len,), word_to_vec_map, word_to_index)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 505)]             0         
                                                                 
 embedding (Embedding)       (None, 505, 50)           20000050  
                                                                 
 lstm (LSTM)                 (None, 505, 128)          91648     
                                                                 
 dropout (Dropout)           (None, 505, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129   

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(messages, y, random_state = 0, test_size = 0.1)

In [28]:
X_train_indices = sentences_to_indices(X_train, word_to_index, max_len)

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

# Example data generation (replace this with your actual data)
# Here we're generating some synthetic data for demonstration.
num_samples = 1000
X = np.random.rand(num_samples, 10)  # Features
Y = np.random.choice(['0', '1'], num_samples)  # Binary labels as strings

# Convert Y to numeric
le = LabelEncoder()
Y = le.fit_transform(Y)  # Convert string labels to integers (0 and 1)

# Split the dataset into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, Y_train, epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
25/25 [==============================] - 0s 1ms/step - loss: 0.7026 - accuracy: 0.5100
Epoch 2/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6947 - accuracy: 0.5125
Epoch 3/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6928 - accuracy: 0.5013
Epoch 4/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6909 - accuracy: 0.5250
Epoch 5/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6892 - accuracy: 0.5412
Epoch 6/50
25/25 [==============================] - 0s 963us/step - loss: 0.6879 - accuracy: 0.5512
Epoch 7/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6873 - accuracy: 0.5462
Epoch 8/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6858 - accuracy: 0.5475
Epoch 9/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6852 - accuracy: 0.5663
Epoch 10/50
25/25 [==============================] - 0s 1ms/step - loss: 0.6842 - accuracy: 0.5700
Epoch 11/50
25/25

In [31]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

7/7 [==============================] - 0s 1ms/step - loss: 0.7015 - accuracy: 0.5200
Test Loss: 0.7015237212181091, Test Accuracy: 0.5199999809265137


## Accuracy of LSTM: 67.25%

In [34]:
model.save("model.h5")

## Predictions

In [35]:
text = "suck it"
text = [clean_text(text)]
text

['suck']

In [36]:
text = sentences_to_indices(text, word_to_index, max_len)

In [37]:
text

array([[23695.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0., 

In [42]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.sequence import pad_sequences
# Define and fit the tokenizer
tokenizer = Tokenizer(num_words=1000)  # Adjust num_words as needed
tokenizer.fit_on_texts(messages)
# Tokenization (this is an example, use your actual tokenizer)
# Assuming tokenizer was created and fit on your training data
sequences = tokenizer.texts_to_sequences([messages])  # Convert text to sequences
padded_sequences = pad_sequences(sequences, maxlen=10)  # Adjust maxlen to your training input size

# Make the prediction
prediction = model.predict(padded_sequences)
print(prediction[0][0])  # Adjust indexing based on your output layer

1.0


## Extras

In [40]:
import pickle

In [41]:
pickle.dump(word_to_index, open('word_to_index.pkl', 'wb'))